In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import jacobi as Jacobi


def chop(X,d=3,suppress=True,eps=1e-10):
    M = np.copy(X)
    w = np.where(np.abs(M) < eps)
    M[w] = 0.0
    np.set_printoptions(precision=d,suppress=suppress,linewidth=10**np.inf)
    print(M)
    print()

$$
\text{P}^{(a,b)}(z) \ = \ \left[ P_{0}^{(a,b)}(z), \ldots,  P_{n}^{(a,b)}(z) , \ldots\right]
$$

<br>

$$
\text{P}^{(a,b)}(-z) \ = \ \text{P}^{(b,a)}(z)\, \Pi
$$

<br>


$$
1 \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a+1,b)}(z)\, A_{a,b}^{+}, \qquad (1-z) \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a-1,b)}(z)\, A_{a,b}^{-}
$$


$$
1 \, \text{P}^{(a-1,b)}(z) \ = \ \text{P}^{(a,b)}(z)\, A_{a-1,b}^{+}, \qquad (1-z) \, \text{P}^{(a+1,b)}(z) \ = \ \text{P}^{(a,b)}(z)\, A_{a+1,b}^{-}
$$



<br>

$$
\left[b + (1+z)\frac{d}{dz} \right] \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a+1,b-1)}(z)\, C_{a,b}^{+}, \qquad
\left[a - (1-z)\frac{d}{dz} \right] \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a-1,b+1)}(z)\, C_{a,b}^{-}
$$


<br>

$$
\frac{d}{dz} \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a+1,b+1)}(z)\, {D}_{a,b}^{+},\qquad
\left[ a(1+z) - b(1-z)   - (1-z^{2})\frac{d}{dz}\right] \, \text{P}^{(a,b)}(z) \ = \ \text{P}^{(a-1,b-1)}(z)\, {D}_{a,b}^{-}
$$




Get some operators

In [ ]:
A  = Jacobi.operator('A')
B  = Jacobi.operator('B')
C  = Jacobi.operator('C')
D  = Jacobi.operator('D')
Z  = Jacobi.operator('Z')
N  = Jacobi.operator('N')
I  = Jacobi.operator('Id')
Pi = Jacobi.operator('Pi')

Check codomains

In [ ]:
print(A(-1).codomain)
print((A(-1)@Pi).codomain)
print((Pi@A(-1)).codomain)
print((Pi@A(-1)@Pi).codomain)

Grid vs coefficient test.

In [ ]:
nab = n,a,b = 14,1,1/2

z,w = Jacobi.quadrature(n,a,b)

P = lambda nab,z : Jacobi.polynomials(*nab,z).T
L = np.diag
d = lambda Op,nab: Op.codomain(*nab)

size = lambda thing: print(np.max(np.abs(thing)))

size(          P(nab, z) - P( d(A(+1),nab), z) @ A(+1)(*nab) )
size(          P(nab, z) - P( d(B(+1),nab), z) @ B(+1)(*nab) )
size( L(1-z) @ P(nab, z) - P( d(A(-1),nab), z) @ A(-1)(*nab) )
size( L(1+z) @ P(nab, z) - P( d(B(-1),nab), z) @ B(-1)(*nab) )
size(          P(nab,-z) - P(    d(Pi,nab), z) @    Pi(*nab) )

We can defie a simple function: 

In [ ]:
f = lambda z: 1 + z + 2*z**2 - z**3 - 2*z**4 - z**5
plt.plot(z,f(z))

The operator function does the right thing:

In [ ]:
F = f(Z)

size( L( f(z) ) @ P(nab,z) - P( d(F,nab), z) @ F(*nab) )

The actual operator:

In [ ]:
chop(F(*nab).todense(),d=3)
F(*nab)

But we can't compute the full matrix-valued function on the grid without some aliasing.

In [ ]:
chop( abs(P( d(F,nab), z).T @ L( w*f(z) ) @ P(nab,z) -  F(*nab) ))

Operator algebra tests

In [ ]:
nab = n,a,b = (400,2,2)

def norm(X,eps=1e-8): 
    print(np.max(np.abs(X(*nab))) < 1e-6)

# I've hijacked the * for the Lie bracket.
# X * Y = X @ Y - Y @ X
M = D(+1) * D(-1)
S = C(+1) * C(-1)

M = A(0) + B(0)
S = A(0) - B(0)

norm(M * S)

for p in (-1,1):
    
    norm( Pi @ C(p) @ Pi - C(-p) )
    norm( Pi @ D(p) @ Pi + D(+p) )
    norm( Pi @ A(p) @ Pi - B(+p) )
    norm( Pi @ B(p) @ Pi - A(+p) )
    
    norm( Z * C(p)  + p * A(p) @ B(-p) )
    norm( Z * D(p)  + p * A(p) @ B(+p) )
    
    norm( M * D(p) - 2*p*D(p) )
    norm( M * C(p)            )
    norm( M * A(p) -   p*A(p) )
    norm( M * B(p) -   p*B(p) )
    
    norm( S * D(p)            )
    norm( S * C(p) - 2*p*C(p) )
    norm( S * A(p) -   p*A(p) )
    norm( S * B(p) +   p*B(p) )
    
    norm( D(p) * C(+p) )
    norm( D(p) * C(-p) )
    
    norm( A(p) * D(+p)           )
    norm( A(p) * D(-p) + p*B(-p) )
    norm( A(p) * C(+p)           )
    norm( A(p) * C(-p) + p*B(+p) )
    
    norm( B(p) * D(+p)           )
    norm( B(p) * D(-p) - p*A(-p) )
    norm( B(p) * C(+p) + p*A(+p) )
    norm( B(p) * C(-p)           )
    
    norm( A(p) * B(+p) )
    norm( A(p) * B(-p) )

Extendable sum test

In [ ]:
DD = D(+1) @ D(-1)
L = DD + Z

print(Z.codomain)
print(DD.codomain)

print(L.codomain)

In [ ]:
t = ((D(+1)@D(-1)).codomain,(D(-1)@D(+1)).codomain)

u = ((A(-1)@A(+1)).codomain,(A(+1)@A(-1)).codomain)

print(t)
print(u)
print(t==u)